## Infraestructuras Computacionales para el Procesamiento de Datos Masivos
### Práctica del Módulo 2: Procesamiento paralelo basado en memoria con Apache Spark
### Autor: Jesús Galán Llano
#### Correo: jgalan279@alumno.uned.es

##### Parte 2: Spark y Machine Learning

Esta segunda parte de la práctica consiste en crear un modelo de Machine Learning utilizando técnicas soportadas por Spark. 

En este caso se ha utilizado un dataset obtenido de la página web Kaggle que contiene datos sobre la calidad del vino tinto "Vinho Verde" portugués (https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009). 

Con estos datos se pretende generar un modelo que prediga la calidad final de vino. Una calidad mayor de 6,5 indica que el vino es considerado "bueno", mientras que un valor inferior categoriza al vino como "malo". 

Debido a estas características se plantea un problema de regresión, es decir, a través de los datos de entrada el modelo deberá predecir la calidad final del vino que permitirá catalogar el vino como "bueno" o "malo".

En primer lugar, importamos las librerías necesarias y creamos los objetos SparkSession y SparkContext.

In [1]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

spark = SparkSession.builder.appName('pt2').getOrCreate()
sc = spark.sparkContext

21/12/11 13:45:29 WARN Utils: Your hostname, jesus-Aspire-A514-52 resolves to a loopback address: 127.0.1.1; using 192.168.1.54 instead (on interface wlp2s0)
21/12/11 13:45:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/11 13:45:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


La versión de Spark utilizada es la 3.2.0. Esto es debido a que es la versión más reciente compatible con Python 3.8.10.

In [2]:
print(f'Running Spark Version {sc.version}')

conf = SparkConf()
print(conf.toDebugString())

Running Spark Version 3.2.0
spark.app.name=pt2
spark.master=local[*]
spark.submit.deployMode=client
spark.submit.pyFiles=
spark.ui.showConsoleProgress=true


En primer lugar, se procede a cargar los datos, incluidos en el fichero winequality-red.csv, en un DataFrame. Debido a que el peso del archivo sobrepasa el límite configurado en el entregador de la práctica no se ha añadido en la entrega, pero se puede descargar desde el siguiente enlace: https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009/download. 
Se añade la opción de inferir el esquema de datos a través de los mismos.

In [3]:
df = spark.read.csv('winequality-red.csv', mode="DROPMALFORMED", inferSchema=True, header=True)

Mostrar el esquema heredado.

In [4]:
df.printSchema()

root
 |-- fixed acidity: double (nullable = true)
 |-- volatile acidity: double (nullable = true)
 |-- citric acid: double (nullable = true)
 |-- residual sugar: double (nullable = true)
 |-- chlorides: double (nullable = true)
 |-- free sulfur dioxide: double (nullable = true)
 |-- total sulfur dioxide: double (nullable = true)
 |-- density: double (nullable = true)
 |-- pH: double (nullable = true)
 |-- sulphates: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- quality: integer (nullable = true)



Como se puede comprobar, todas las variables son tipos numéricos, lo que facilita enormemente la limpieza y la transformación de datos para el futuro entrenamiento del modelo. 

Las variables de entrada del modelo se corresponden con: fixed acidity, volatile acidity, citric acid, residual sugar, chlorides, free sulfur dioxide, total sulfur dioxide, density, pH y sulphates. En el enlace anterior se proporciona más información acerca de qué representa cada una de estas características. 

La única variable de salida es la variable quality, que representa la calidad final del vino.

Mostrar el número de registros del dataset.


In [5]:
df.count()

1599

Mostrar una muestra de los datos.

In [6]:
df.show(n=5, truncate=True, vertical=True)

-RECORD 0----------------------
 fixed acidity        | 7.4    
 volatile acidity     | 0.7    
 citric acid          | 0.0    
 residual sugar       | 1.9    
 chlorides            | 0.076  
 free sulfur dioxide  | 11.0   
 total sulfur dioxide | 34.0   
 density              | 0.9978 
 pH                   | 3.51   
 sulphates            | 0.56   
 alcohol              | 9.4    
 quality              | 5      
-RECORD 1----------------------
 fixed acidity        | 7.8    
 volatile acidity     | 0.88   
 citric acid          | 0.0    
 residual sugar       | 2.6    
 chlorides            | 0.098  
 free sulfur dioxide  | 25.0   
 total sulfur dioxide | 67.0   
 density              | 0.9968 
 pH                   | 3.2    
 sulphates            | 0.68   
 alcohol              | 9.8    
 quality              | 5      
-RECORD 2----------------------
 fixed acidity        | 7.8    
 volatile acidity     | 0.76   
 citric acid          | 0.04   
 residual sugar       | 2.3    
 chlorid

Limpiar el dataset eliminando las filas con elementos nulos.

In [7]:
df = df.dropna()

In [8]:
df.count()

1599

Limpiar el dataset eliminando las filas con elementos duplicados.

In [9]:
df = df.dropDuplicates()

In [10]:
df.count()

1359

A continuación, realizamos un estudio de las variables de entrada del modelo. De esta forma se podrán analizar los datos para determinar si son útiles para crear un modelo fiable.

In [11]:
numeric_features = [t[0] for t in df.dtypes if t[1] == 'int' or t[1] == 'double']
df.select(numeric_features).describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
fixed acidity,1359,8.310596026490076,1.7369898075324683,4.6,15.9
volatile acidity,1359,0.5294775570272255,0.18303131761907196,0.12,1.58
citric acid,1359,0.27233259749816047,0.1955365445504638,0.0,1.0
residual sugar,1359,2.5233995584988946,1.3523137577104223,0.9,15.5
chlorides,1359,0.08812362030905024,0.049376862443486075,0.012,0.611
free sulfur dioxide,1359,15.893303899926417,10.447270259048686,1.0,72.0
total sulfur dioxide,1359,46.82597498160412,33.408945706616564,6.0,289.0
density,1359,0.9967089477557036,0.001868917132559179,0.9900700000000001,1.00369
pH,1359,3.3097866077998535,0.1550363112872961,2.74,4.01


Como se puede observar en la tabla anterior, la media de la clase quality es menor de 6.5, por lo que podemos deducir que los datos están desbalanceados, hay más vinos considerados "malos" que "buenos".

Seguidamente, creamos un objeto VectorAssembler para unir varias columnas en una única. Estas columnas se corresponden con las columnas de entrada del modelo. 

In [12]:
assembler = VectorAssembler(
    inputCols=["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide",
    "total sulfur dioxide","density","pH","sulphates","alcohol"],
    outputCol="features")

Se dividen los datos de entrada en subconjuntos de prueba y de entrenamiento. Para evitar que el modelo esté sobreentrenado se establece que el 75% de los datos formen parte del subconjunto de entrenamiento y el restante 25% formen el subconjunto de test.

In [13]:
train_df, test_df = df.randomSplit([0.75,0.25], seed=80)
print("Training Dataset Count: " + str(train_df.count()))
print("Test Dataset Count: " + str(test_df.count()))

Training Dataset Count: 1026
Test Dataset Count: 333


Como se ha comentado anteriormente, el problema es tratado como un problema de regresión lineal. Dentro de esta familia de algortimos se ha elegido el método gausiano porque el dato que precide es un valor continuo al igual que el valor con el que se mide la calidad del vino, que es la característica que se pretende predecir (https://spark.apache.org/docs/latest/ml-classification-regression.html#linear-regression.)

In [14]:
glr = GeneralizedLinearRegression(family="gaussian", link="Identity", featuresCol="features", labelCol="quality",maxIter=10, regParam=0.3)

Con el fin de utilizar distintos transformadores y estimadores juntos utilizamos el método Pipeline que se encarga de crear un flujo de trabajo o workflow. 

In [15]:
pipeline = Pipeline(stages=[assembler, glr])

A continuación, entrenamos el modelo con los datos de entrenamiento.

In [16]:
model = pipeline.fit(train_df)

Tras esto, se generan las predicciones utilizando el modelo generado y los datos de test.

In [17]:
predictions = model.transform(test_df)

Por último, se evalua el modelo creado obteniendo el error cuadrático medio (RMSE).

In [18]:
evaluator = RegressionEvaluator(labelCol="quality",predictionCol="prediction", metricName="mse")
rmse = evaluator.evaluate(predictions)

print(f'RMSE: {rmse}')

RMSE: 0.4936894482155893


El RMSE mide la cantidad de error que hay entre dos conjuntos de datos, los valores predichos y los valores conocidos. Cuanto más pequeño es este valor más cercanos son los valores predichos y observados. Por tanto, se podría considerar que el modelo creado es suficientemente bueno con un RMSE del 0.4937.

Además, se calculan otras métricas que son útiles para la evaluación del modelo generado.

In [19]:
str_model = model.stages[-1]
print("Coefficients: " + str(str_model.coefficients))
print("Intercept: " + str(str_model.intercept))

Coefficients: [0.018178444087013024,-0.7824214756567364,0.17941567981650883,0.002222012200290806,-1.613248711982213,0.0015791410285770024,-0.0022784230175551534,-38.68046925529207,-0.16794392530541905,0.7485294052811008,0.19572528081694826]
Intercept: 42.619657854273136


In [20]:
summary = str_model.summary
print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
print("T Values: " + str(summary.tValues))
print("P Values: " + str(summary.pValues))
print("Dispersion: " + str(summary.dispersion))
print("Null Deviance: " + str(summary.nullDeviance))
print("Residual Degree Of Freedom Null: " + str(summary.residualDegreeOfFreedomNull))
print("Deviance: " + str(summary.deviance))
print("Residual Degree Of Freedom: " + str(summary.residualDegreeOfFreedom))
print("AIC: " + str(summary.aic))
print("Deviance Residuals: ")
summary.residuals().show()

Coefficient Standard Errors: [0.013774958285278863, 0.1078562350567655, 0.11539488465751785, 0.013920032345109471, 0.4118763892939464, 0.001931787141533024, 0.0006109783907734636, 12.298768555801455, 0.13969837708336016, 0.11290054473983274, 0.018727672459651207, 12.250502493258121]
T Values: [1.3196732585709616, -7.254299904358263, 1.5547975141965713, 0.15962694232326738, -3.9168273635391024, 0.8174508436389274, -3.729138463752868, -3.1450684741153285, -1.2021895229692205, 6.6299893149852105, 10.451126867935063, 3.479013034585987]
P Values: [0.18724199830291122, 8.01136934569513e-13, 0.12030634062577117, 0.8732067440407345, 9.572822579939633e-05, 0.4138628574893837, 0.00020277687469860695, 0.0017088068235480147, 0.2295707741292583, 5.447375883704808e-11, 0.0, 0.0005246524791024942]
Dispersion: 0.432361561837038
Null Deviance: 684.6978557504877
Residual Degree Of Freedom Null: 1025
Deviance: 438.41462370275656
Residual Degree Of Freedom: 1014
AIC: 2065.2972301936197
Deviance Residuals:

Finalmente, se guarda el modelo creado con el fin de poder utilizarlo en el futuro para predecir nuevos conjuntos de datos. 

In [21]:
model.write().overwrite().save("models/red-wine")

In [22]:
spark.stop()

## Conclusiones

A modo de conclusiones, me gustaría comentar que esta segunda práctica de la asignatura me ha ayudado a comprender mejor la utilización de Apache Spark así como sus características. Este módulo era uno de los que más me interesaban en el máster y he terminado satisfecho con todos los conocimientos que he adquirido. Además, la bibliografía recomendada es muy extensa y gracias a su calidad he podido ampliar varios conceptos. 

La primera parte de Spark SQL me ha resultado sencilla porque tengo conocimientos previos de SQL en el grado de Ingeniería Informática. Por otra parte, me hubiera gustado que se le hubiera dado más importancia a la parte de Machine Learning con Spark porque pienso que actualmente está muy demandada debido a sus aplicaciones en 
diversos sectores. Quizá se podría equilibrar el porcentaje de peso entre las dos partes de la práctica y extender los recursos proporcionados por el equipo docente en la parte de Machine Learning.